In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/NLPP/Univ_NLP')
os.getcwd()

'/content/drive/MyDrive/Colab Notebooks/NLPP/Univ_NLP'

In [4]:
import pandas as pd
train_dataset = pd.read_json("df_all.json", orient="records", lines=True)

print(f"전체 데이터셋 개수: {len(train_dataset)}")
train_dataset.head()

전체 데이터셋 개수: 5844


,info.gender,info.ageRange,info.experience,question.raw.text,answer.summary.text,category
0,1,0,0,지원자님이 가장 존경하는 역사 속 인물을 말씀해 주십시오 그리고 그 인물을 왜 존경...,저는 성경에 나오는 인물인 모세를 개인적으로 가장 존경합니다. 본인의 능력을 최대한...,1
1,1,0,0,지원자님께서 지금까지 경험하신 여러 대외 활동 중에 무엇을 배웠고 어떤 자세로 임했...,지금까지 했던 대외 활동들은 대부분 기업의 홍보 대사나 마케팅 활동을 많이 했습니다...,3
2,1,0,0,언제부터 아이씨티에 관심이 있었는지 그리고 지금과 같은 아이씨티 역량을 갖추기 위해...,이 회사에 맞는 인재가 되기 위해 프로그래밍 언어를 공부했습니다. 고난과 역경이 많...,4
3,1,0,0,지원자님이 다른 지원자 분들에 비해서 가지고 있는 차별성이 있다면 혹은 장점을 어필...,아이씨티 분야에서 남들보다 호기심이 많고 새로운 기술을 습득하는 것이 빠른 게 장점...,1
4,1,0,0,지원자님에 장점과 단점을 말씀해 주시고 그 장점을 특화시키고 단점을 극복하기 위해 ...,저는 유연성이 강해서 어떤 상황에서도 잘 적응하고 인간관계에서도 유연성이 있습니다....,2


In [5]:
"""
# info.gender -> MALE=0, FEMALE=1
# info.ageRange -> ['-34', '35-44', '55-', '45-54'] -> 0, 1, 2, 3
# info.experience -> ['NEW', 'EXPERIENCED'] -> 0, 1
category_range_map = {'\'technology\'': 0, '\'attitude\'': 1, '\'background\'': 2, '\'personality\'': 3, '\'etc\'': 4}
"""

"\n# info.gender -> MALE=0, FEMALE=1\n# info.ageRange -> ['-34', '35-44', '55-', '45-54'] -> 0, 1, 2, 3\n# info.experience -> ['NEW', 'EXPERIENCED'] -> 0, 1\ncategory_range_map = {''technology'': 0, ''attitude'': 1, ''background'': 2, ''personality'': 3, ''etc'': 4}\n"

In [6]:
!pip3 install kobert-transformers

In [7]:
#kobert transformers 정상작동 확인 예제코드
import torch
from kobert_transformers import get_kobert_model, get_distilkobert_model

model = get_kobert_model()
model.eval() # 학습 안하고 평가

input_ids = torch.LongTensor([[31, 51, 99], [15, 5, 0]])
attention_mask = torch.LongTensor([[1, 1, 1], [1, 1, 0]])
token_type_ids = torch.LongTensor([[0, 0, 1], [0, 1, 0]])
output = model(input_ids, attention_mask, token_type_ids)

output

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/426 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.2461,  0.2428,  0.2590,  ..., -0.4861, -0.0731,  0.0756],
         [-0.2478,  0.2420,  0.2552,  ..., -0.4877, -0.0727,  0.0754],
         [-0.2472,  0.2420,  0.2561,  ..., -0.4874, -0.0733,  0.0765]],

        [[ 0.0768, -0.1234,  0.1534,  ..., -0.2518, -0.2571,  0.1602],
         [-0.2419, -0.2821,  0.1962,  ..., -0.0172, -0.2960,  0.3679],
         [ 0.0911, -0.1437,  0.3412,  ...,  0.2526, -0.1780,  0.2619]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-0.0903, -0.0444,  0.1579,  ...,  0.1010, -0.0819,  0.0529],
        [ 0.0742, -0.0116, -0.6845,  ...,  0.0024, -0.0447,  0.0122]],
       grad_fn=<TanhBackward0>), hidden_states=None, past_key_values=None, attentions=None, cross_attentions=None)

In [8]:
sequence_output = output[0]  # Last hidden state
pooled_output = output[1]    # Pooler output

print(f"sequence_output is {sequence_output}") # last_hidden_state
print(f"pooled_output is {pooled_output}") # pooler_output

from kobert_transformers import get_tokenizer

tokenizer = get_tokenizer()
# 문장을 토큰화(분절) 한다.
tokenizer.tokenize("[CLS] 한국어 모델을 공유합니다. [SEP]") #['[CLS]', '▁한국', '어', '▁모델', '을', '▁공유', '합니다', '.', '[SEP]']
# 내가 입력들의 어떤 인덱스로 들어가는지 알 수 있다.
tokenizer.convert_tokens_to_ids(['[CLS]', '▁한국', '어', '▁모델', '을', '▁공유', '합니다', '.', '[SEP]']) #[2, 4958, 6855, 2046, 7088, 1050, 7843, 54, 3]

sequence_output is tensor([[[-0.2461,  0.2428,  0.2590,  ..., -0.4861, -0.0731,  0.0756],
         [-0.2478,  0.2420,  0.2552,  ..., -0.4877, -0.0727,  0.0754],
         [-0.2472,  0.2420,  0.2561,  ..., -0.4874, -0.0733,  0.0765]],

        [[ 0.0768, -0.1234,  0.1534,  ..., -0.2518, -0.2571,  0.1602],
         [-0.2419, -0.2821,  0.1962,  ..., -0.0172, -0.2960,  0.3679],
         [ 0.0911, -0.1437,  0.3412,  ...,  0.2526, -0.1780,  0.2619]]],
       grad_fn=<NativeLayerNormBackward0>)
pooled_output is tensor([[-0.0903, -0.0444,  0.1579,  ...,  0.1010, -0.0819,  0.0529],
        [ 0.0742, -0.0116, -0.6845,  ...,  0.0024, -0.0447,  0.0122]],
       grad_fn=<TanhBackward0>)


tokenizer_config.json:   0%|          | 0.00/263 [00:00<?, ?B/s]

tokenizer_78b3253a26.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

[2, 4958, 6855, 2046, 7088, 1050, 7843, 54, 3]

모델 구현

In [9]:
data = train_dataset

# KoBERT 모델 및 토크나이저 로드
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
model = get_kobert_model()
model.eval()

# 임베딩 추출 함수
def get_cls_token(sentence):
    tokenized_sent = tokenizer(sentence, return_tensors='pt', truncation=True, max_length=128)
    input_ids = tokenized_sent['input_ids']
    attention_mask = tokenized_sent['attention_mask']
    token_type_ids = tokenized_sent['token_type_ids']
    with torch.no_grad():
        output = model(input_ids, attention_mask, token_type_ids)
    return output[1].detach().cpu().numpy()

# 데이터 벡터화
data['answer_embedding'] = data['answer.summary.text'].apply(get_cls_token)

NameError: name 'KoBERTTokenizer' is not defined

In [ ]:
import random
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# 예시로 사용한 데이터 (이전 코드와 동일)
category = 1  # 사용자 입력
filtered_data = data[data['category'] == category]
random_question = random.choice(filtered_data['question.raw.text'])

# 사용자 답변 입력
user_answer = "모세는 정말 대단한 인물이라고 생각합니다."

# 유사도 계산
user_embedding = get_cls_token(user_answer)
embeddings = np.vstack(filtered_data['answer_embedding'])
similarities = cosine_similarity(user_embedding, embeddings)

# 가장 유사한 답변 찾기
best_match_idx = np.argmax(similarities)
predicted_answer = filtered_data.iloc[best_match_idx]['answer.summary.text']

# 유사도 값
similarity_score = similarities[0, best_match_idx]

# 유사도 값에 따른 메시지
if similarity_score >= 0.8:
    similarity_message = f"일치합니다! 유사도는 {similarity_score:.2f} 입니다."
else:
    similarity_message = f"일치하지 않습니다. 유사도는 {similarity_score:.2f} 입니다."

# 결과 출력
print(f"질문: {random_question}")
print(f"사용자 답변: {user_answer}")
print(f"예측 답변: {predicted_answer}")
print(similarity_message)

기존 코드

In [ ]:
"""

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def get_cls_token(sentence):
  model.eval()
  tokenized_sent = tokenizer(
      sentence,
      return_tensors = 'pt',
      truncation = True,
      add_special_tokens = True,
      max_length = 128
  )

  input_ids = tokenized_sent['input_ids']
  attention_mask = tokenized_sent['attention_mask']
  token_type_ids = tokenized_sent['token_type_ids']

  with torch.no_grad():
    output = model(input_ids, attention_mask, token_type_ids)

  cls_output = output[1]
  cls_token = cls_output.detach().cpu().numpy()
  return cls_token

In [ ]:
get_cls_token('너 요즘 바뻐?')

In [ ]:
def predict(query, candidates):
  candidates_cls = []

  for cand in candidates:
    cand_cls = get_cls_token(cand)
    candidates_cls.append(cand_cls)

  candidates_cls = np.array(candidates_cls).squeeze(axis=1)
  query_cls = get_cls_token(query)
  similarity_list = cosine_similarity(query_cls, candidates_cls)
  target_ids = np.argmax(similarity_list)
  return candidates[target_ids]

In [ ]:
samle_query = '너 요즘 바뻐?'
print(get_cls_token(samle_query).shape)

In [ ]:
samle_query = '너 요즘 바뻐?'
sample_candidates = ['바쁘면 가라','아니 안바뻐','오늘은 이만','에붸붸붸']

predicted_answer = predict('samle_query', sample_candidates)
print(f'결과: {predicted_answer}')

In [ ]:
samle_query = '힘든 연애 좋은 연애라는게 무슨 차이일까?'
sample_candidates = random.sample(answer_list,100)

predicted_answer = predict('samle_query', sample_candidates)
print(f'결과: {predicted_answer}')

In [ ]:
end = 1
while end == 1:
  sentence = input('질문을 입력하세요:')
  if len(sentence) == 0:
    break
  predicted_answer = predict (sentence, response_candidates)
  print (predicted_answer)
  print('Wn')